# Azure AI Search con Embeddings Pre-calculados

Este notebook utiliza los datos generados en el módulo anterior (04-document-ingestion) que ya incluyen:
- Embeddings vectoriales
- Categorías
- Tags

A diferencia del enfoque con Skillset, aquí NO necesitamos generar embeddings durante la ingesta,
simplemente indexamos los vectores que ya existen.


In [ ]:
from __future__ import annotations

import json
import os
import pathlib
import time
from typing import Any, Dict, List

from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import ResourceExistsError
from azure.storage.blob import BlobServiceClient

from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
from azure.search.documents.models import VectorizedQuery
from azure.search.documents.indexes.models import (
    HnswAlgorithmConfiguration,
    IndexingParameters,
    SearchField,
    SearchFieldDataType,
    SearchIndex,
    SearchIndexer,
    SearchIndexerDataContainer,
    SearchIndexerDataSourceConnection,
    SimpleField,
    SearchableField,
    VectorSearch,
    VectorSearchProfile,
)
from dotenv import load_dotenv
import openai


## 1. Configuración


In [ ]:
load_dotenv(override=True)

# Azure Search
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
AZURE_SEARCH_KEY = os.getenv("AZURE_SEARCH_KEY")

# Azure Storage
AZURE_STORAGE_CONNECTION_STRING = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
BLOB_CONTAINER_NAME = os.getenv("AZURE_STORAGE_CONTAINER", "documents-precomputed-vectors")
BLOB_PREFIX = "json-chunks"

# Nombres de recursos
INDEX_NAME = "chunks-precomputed-vectors-index"
DATA_SOURCE_NAME = "chunks-blob-datasource"
INDEXER_NAME = "chunks-blob-indexer"
VECTOR_PROFILE_NAME = "chunksProfile"
ALGORITHM_NAME = "chunksHnsw"

# Archivo de datos del módulo anterior
DATA_FILE = pathlib.Path("..") / "04-document-ingestion" / "data" / "rag_ingested_chunks.json"

print("✓ Configuración cargada")
print(f"  - Archivo de datos: {DATA_FILE}")
print(f"  - Índice: {INDEX_NAME}")
print(f"  - Contenedor: {BLOB_CONTAINER_NAME}")


## 2. Inicializar Clientes


In [ ]:
# Validar variables de entorno
required_vars = [
    "AZURE_SEARCH_ENDPOINT",
    "AZURE_SEARCH_KEY",
    "AZURE_STORAGE_CONNECTION_STRING",
    "GITHUB_TOKEN",
]

missing = [var for var in required_vars if os.getenv(var) is None]
if missing:
    raise EnvironmentError(f"Faltan variables de entorno: {', '.join(missing)}")

# Inicializar clientes de Azure Search
credential = AzureKeyCredential(AZURE_SEARCH_KEY)
index_client = SearchIndexClient(endpoint=AZURE_SEARCH_ENDPOINT, credential=credential)
indexer_client = SearchIndexerClient(endpoint=AZURE_SEARCH_ENDPOINT, credential=credential)
search_client = SearchClient(endpoint=AZURE_SEARCH_ENDPOINT, index_name=INDEX_NAME, credential=credential)

# Cliente de OpenAI (GitHub Models) para generar embeddings en las queries
openai_client = openai.OpenAI(
    base_url="https://models.github.ai/inference",
    api_key=os.environ.get("GITHUB_TOKEN")
)

print("✓ Clientes inicializados")


## 3. Cargar Documentos con Embeddings Pre-calculados


In [ ]:
if not DATA_FILE.exists():
    raise FileNotFoundError(f"No se encuentra el archivo: {DATA_FILE}")

with open(DATA_FILE, "r", encoding="utf-8") as f:
    documents = json.load(f)

print(f"✓ Cargados {len(documents)} documentos")
print(f"\nEjemplo del primer documento:")
sample_doc = documents[0].copy()
# Truncar embedding para visualización
if 'embedding' in sample_doc:
    embedding_len = len(sample_doc['embedding'])
    sample_doc['embedding'] = f"[vector de {embedding_len} dimensiones]"
print(json.dumps(sample_doc, indent=2, ensure_ascii=False))


## 4. Subir Documentos a Blob Storage


In [ ]:
print("\n🚚 Subiendo documentos al contenedor de Blob Storage...")
blob_service = BlobServiceClient.from_connection_string(AZURE_STORAGE_CONNECTION_STRING)
container_client = blob_service.get_container_client(BLOB_CONTAINER_NAME)

try:
    container_client.create_container()
    print(f"  ✓ Contenedor '{BLOB_CONTAINER_NAME}' creado")
except ResourceExistsError:
    print(f"  • Contenedor '{BLOB_CONTAINER_NAME}' ya existe, se reutiliza")

uploaded_count = 0
for doc in documents:
    blob_name = f"{BLOB_PREFIX}/{doc['id']}.json"
    data = json.dumps(doc, ensure_ascii=False).encode("utf-8")
    container_client.upload_blob(name=blob_name, data=data, overwrite=True)
    uploaded_count += 1
    if uploaded_count % 100 == 0:
        print(f"  • Subidos {uploaded_count}/{len(documents)} documentos...")

print(f"  ✓ {uploaded_count} documentos subidos correctamente")


## 5. Crear Índice de Azure AI Search

El índice acepta vectores pre-calculados. No necesitamos vectorizer para la ingesta.


In [ ]:
print("\n🧱 Creando índice para vectores pre-calculados...")

# Definir campos del índice
fields = [
    SimpleField(
        name="id", 
        type=SearchFieldDataType.String, 
        key=True, 
        filterable=True, 
        sortable=True
    ),
    SearchableField(
        name="content", 
        type=SearchFieldDataType.String, 
        searchable=True
    ),
    SimpleField(
        name="category", 
        type=SearchFieldDataType.String, 
        filterable=True, 
        facetable=True, 
        sortable=True
    ),
    SimpleField(
        name="source", 
        type=SearchFieldDataType.String, 
        filterable=True, 
        facetable=True
    ),
    SimpleField(
        name="tags",
        type=SearchFieldDataType.Collection(SearchFieldDataType.String),
        filterable=True,
        facetable=True,
    ),
    SearchField(
        name="embedding",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=1536,
        vector_search_profile_name=VECTOR_PROFILE_NAME,
    ),
]

# Configurar búsqueda vectorial
vector_search = VectorSearch(
    profiles=[
        VectorSearchProfile(
            name=VECTOR_PROFILE_NAME,
            algorithm_configuration_name=ALGORITHM_NAME,
        )
    ],
    algorithms=[HnswAlgorithmConfiguration(name=ALGORITHM_NAME)],
)

index = SearchIndex(name=INDEX_NAME, fields=fields, vector_search=vector_search)

# Eliminar índice anterior si existe
try:
    index_client.delete_index(INDEX_NAME)
    print("  • Índice anterior eliminado")
except Exception:
    pass

index_client.create_index(index)
print(f"  ✓ Índice '{INDEX_NAME}' creado")


## 6. Crear Data Source


In [ ]:
print("\n🔌 Creando data source del contenedor...")

# Limpieza preventiva del indexer
try:
    indexer_client.delete_indexer(INDEXER_NAME)
except Exception:
    pass

container = SearchIndexerDataContainer(name=BLOB_CONTAINER_NAME, query=f"{BLOB_PREFIX}")
data_source = SearchIndexerDataSourceConnection(
    name=DATA_SOURCE_NAME,
    type="azureblob",
    connection_string=AZURE_STORAGE_CONNECTION_STRING,
    container=container,
    description="Blob container con chunks y embeddings pre-calculados",
)

try:
    indexer_client.delete_data_source_connection(DATA_SOURCE_NAME)
except Exception:
    pass

indexer_client.create_data_source_connection(data_source)
print(f"  ✓ Data source '{DATA_SOURCE_NAME}' creada")


## 7. Crear Indexer (SIN Skillset)

Como los embeddings ya están calculados, NO necesitamos un skillset.
El indexer simplemente lee los campos del JSON y los mapea al índice.


In [ ]:
print("\n⚙️  Creando indexer (sin skillset, vectores pre-calculados)...")

indexer = SearchIndexer(
    name=INDEXER_NAME,
    data_source_name=DATA_SOURCE_NAME,
    target_index_name=INDEX_NAME,
    description="Indexer para chunks con embeddings pre-calculados",
    parameters=IndexingParameters(configuration={"parsingMode": "json"}),
)

try:
    indexer_client.delete_indexer(INDEXER_NAME)
except Exception:
    pass

indexer_client.create_indexer(indexer)
print(f"  ✓ Indexer '{INDEXER_NAME}' creado")


## 8. Ejecutar Indexer y Esperar


In [ ]:
print("\n▶️  Ejecutando indexer y esperando a que finalice...")
indexer_client.run_indexer(INDEXER_NAME)

timeout_seconds = 300
waited = 0
poll_interval = 5

while waited < timeout_seconds:
    status = indexer_client.get_indexer_status(INDEXER_NAME)
    last_result = status.last_result
    
    if last_result and last_result.status == "success":
        print(f"  ✓ Indexer completado exitosamente")
        print(f"    - Documentos procesados: {last_result.item_count}")
        print(f"    - Documentos fallidos: {last_result.failed_item_count}")
        if last_result.failed_item_count > 0:
            print(f"    ⚠️  Errores: {last_result.error_message}")
        break
    elif last_result and last_result.status == "transientFailure":
        print(f"  ⚠️ Fallo transitorio: {last_result.error_message}")
    elif last_result and last_result.status == "error":
        raise RuntimeError(f"Indexer falló: {last_result.error_message}")
    
    print(f"  • Estado: {status.status} (esperados {waited}/{timeout_seconds}s)")
    time.sleep(poll_interval)
    waited += poll_interval
else:
    raise TimeoutError("El indexer no completó dentro del tiempo esperado")


## 9. Función auxiliar para generar embeddings de queries


In [ ]:
def generate_query_embedding(text: str) -> List[float]:
    """
    Genera el embedding de una query usando text-embedding-3-small.
    """
    response = openai_client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    return response.data[0].embedding

print("✓ Función de embedding lista")


## 10. Búsqueda Vectorial Pura


In [ ]:
def vector_search(query: str, top_k: int = 3) -> None:
    print(f"\n🔎 Búsqueda vectorial: '{query}'")
    
    # Generar embedding de la query
    query_vector = generate_query_embedding(query)
    
    # Crear consulta vectorizada
    vector_query = VectorizedQuery(
        vector=query_vector,
        k_nearest_neighbors=top_k,
        fields="embedding"
    )
    
    # Ejecutar búsqueda
    results = search_client.search(
        search_text=None,
        vector_queries=[vector_query],
        select=["id", "content", "category", "tags"],
        top=top_k
    )
    
    print(f"\nResultados:")
    found = False
    for i, result in enumerate(results, 1):
        found = True
        snippet = result["content"][:150].replace("\n", " ")
        score = result.get("@search.score", 0.0)
        category = result.get("category", "N/A")
        tags = result.get("tags", [])
        
        print(f"\n{i}. Score: {score:.4f}")
        print(f"   ID: {result['id']}")
        print(f"   Categoría: {category}")
        print(f"   Tags: {', '.join(tags) if tags else 'N/A'}")
        print(f"   Contenido: {snippet}...")
    
    if not found:
        print("  (No se encontraron resultados)")

# Realizar búsquedas de ejemplo
vector_search("¿Qué museos puedo visitar en Bilbao?")
vector_search("Gastronomía y pintxos")
vector_search("Cómo llegar y transporte")


## 11. Búsqueda Híbrida (Texto + Vector)


In [ ]:
def hybrid_search(query: str, top_k: int = 3) -> None:
    print(f"\n🔎 Búsqueda híbrida (texto + vector): '{query}'")
    
    # Generar embedding de la query
    query_vector = generate_query_embedding(query)
    
    # Crear consulta vectorizada
    vector_query = VectorizedQuery(
        vector=query_vector,
        k_nearest_neighbors=top_k,
        fields="embedding"
    )
    
    # Ejecutar búsqueda híbrida (texto + vector)
    results = search_client.search(
        search_text=query,  # Búsqueda de texto también
        vector_queries=[vector_query],
        select=["id", "content", "category", "tags"],
        top=top_k
    )
    
    print(f"\nResultados:")
    found = False
    for i, result in enumerate(results, 1):
        found = True
        snippet = result["content"][:150].replace("\n", " ")
        score = result.get("@search.score", 0.0)
        category = result.get("category", "N/A")
        tags = result.get("tags", [])
        
        print(f"\n{i}. Score: {score:.4f}")
        print(f"   ID: {result['id']}")
        print(f"   Categoría: {category}")
        print(f"   Tags: {', '.join(tags) if tags else 'N/A'}")
        print(f"   Contenido: {snippet}...")
    
    if not found:
        print("  (No se encontraron resultados)")



## 12. Búsqueda con Filtros (por categoría y tags)


In [ ]:
def filtered_search(query: str, category: str = None, tags: List[str] = None, top_k: int = 3) -> None:
    filters = []
    if category:
        filters.append(f"category eq '{category}'")
    if tags:
        tag_filters = [f"tags/any(t: t eq '{tag}')" for tag in tags]
        filters.extend(tag_filters)
    
    filter_str = " and ".join(filters) if filters else None
    
    print(f"\n🔎 Búsqueda con filtros: '{query}'")
    if filter_str:
        print(f"   Filtros: {filter_str}")
    
    # Generar embedding de la query
    query_vector = generate_query_embedding(query)
    
    # Crear consulta vectorizada
    vector_query = VectorizedQuery(
        vector=query_vector,
        k_nearest_neighbors=50,  # Buscar más candidatos antes de filtrar
        fields="embedding"
    )
    
    # Ejecutar búsqueda con filtros
    results = search_client.search(
        search_text=query,
        vector_queries=[vector_query],
        filter=filter_str,
        select=["id", "content", "category", "tags"],
        top=top_k
    )
    
    print(f"\nResultados:")
    found = False
    for i, result in enumerate(results, 1):
        found = True
        snippet = result["content"][:150].replace("\n", " ")
        score = result.get("@search.score", 0.0)
        category = result.get("category", "N/A")
        tags = result.get("tags", [])
        
        print(f"\n{i}. Score: {score:.4f}")
        print(f"   ID: {result['id']}")
        print(f"   Categoría: {category}")
        print(f"   Tags: {', '.join(tags) if tags else 'N/A'}")
        print(f"   Contenido: {snippet}...")
    
    if not found:
        print("  (No se encontraron resultados)")

## 13. Ejemplos de búsqueda con filtros


In [ ]:
filtered_search("información turística", category="Bilbao")

filtered_search("museos", tags=["museos"])


## 14. Realizar búsquedas híbridas


In [ ]:
hybrid_search("Guggenheim")

hybrid_search("running y deportes")
